In [1]:
import requests
import json
import re
# import pandas as pd
from collections import defaultdict, deque
from datetime import datetime

In [2]:
from datetime import datetime, timedelta, date
from time import sleep
from random import randrange
from pathlib import Path
from fast_flights import FlightData, Passengers, create_filter, get_flights, FlightsAPIResult

MAX_WEEKS = 20

search_pairs = [
    ['LAS', 'LAX'],
    ['SEA', 'YYZ'],
    ['NYC', 'TYO'],
    ['SAN', 'EZE'],
    ['SNA', 'BOS'],
]

def random_datestr() -> str:
    rand = randrange(1, MAX_WEEKS)
    return (date.today() + timedelta(days=round(rand * 7))).strftime('%Y-%m-%d')

def random_daterange(days_between: int = 0) -> tuple[date, date]:
    rand = randrange(1, MAX_WEEKS)
    start = date.today() + timedelta(days=round(rand * 7))
    return start, start + timedelta(days=days_between)

common_filter_kwargs = {
    'trip': 'one-way',
    'seat': 'economy',
    'passengers': Passengers(
        adults=1,
        children=0,
        infants_in_seat=0,
        infants_on_lap=0
    ),
}

# Create a new filter
filters = [create_filter(flight_data=[FlightData(date=random_datestr(), from_airport=from_, to_airport=to)], **common_filter_kwargs)
           for from_, to in search_pairs]


In [3]:
# Get flights with a filter
results = []
for filter_ in filters:
    print('requesting flight')
    results.append(get_flights(filter_))
    sleep(randrange(8, 20))

# write to json file if success
Path('results.json').write_text(json.dumps([result.raw_data.raw_data for result in results]))

requesting flight


KeyboardInterrupt: 

In [4]:
raw_results = json.loads(Path('results.json').read_text())
results = [FlightsAPIResult.parse(result) for result in raw_results]

We want to find real list types in this data schema (as opposed to a packed struct encoded as a list). 

In order to do so, we can scan multiple instances of the data, and track the size of each list we encounter (using the path/indices). When the size of a list is not constant, we know that the data type of that index is a list type rather than a struct packed as a list.

The length of a struct packed as a list should never change because the struct is well defined.

In [125]:
values = defaultdict(lambda: set())
num_elements = defaultdict(lambda: set())

# determine which keys refer to list types
def find_lists(data: list, prefix: str = '') -> bool:
    for i, value in enumerate(data):
        key_str = f'{prefix}{i}'
        if not isinstance(value, list):
            values[key_str].add(value)
            continue
        num_elements[key_str].add(len(value))
        find_lists(value, prefix=f'{key_str},')

# scan all results to populate values and num_elements
# for result in results:
#     find_lists(result.raw_data.raw_data)

find_lists(results[3].raw_data.raw_data[3])

# sort so we go in breadth first order
all_list_keys = sorted(num_elements.keys(), key=lambda index: len(index))

# num_elements with keys fixed to denote real arrays, an index of i is used where the previous index is referring to a real array type
fixed_num_elements = defaultdict(lambda: set())

# set of keys that denote real arrays
real_list_keys = ['0']

for list_key in all_list_keys:
    # fix the list key by substituting array indices with i, IE: (0,2,0), (0,2,1), (0,2,2) -> (0,2,i)
    fixed_list_key = list_key
    for real_list_key in real_list_keys:
        # modify all keys starting with list_key to use i as the next key rather than whatever index it was
        if fixed_list_key.startswith(real_list_key):
            fixed_list_key = re.sub(rf'({real_list_key}),\d+(,?.*)', r'\1,i\2', fixed_list_key)

    # if the size of the list at the same index isn't constant, this is a real list type
    if (fixed_list_key in fixed_num_elements and len(fixed_num_elements[fixed_list_key]) > 1) or len(num_elements[fixed_list_key]) > 1:
        if fixed_list_key not in real_list_keys:
            print(f'found real list {fixed_list_key}')
            real_list_keys.append(fixed_list_key)
    else:
        # merge num_elements
        if fixed_list_key in fixed_num_elements:
            fixed_num_elements[fixed_list_key] |= num_elements[list_key]
        else:
            fixed_num_elements[fixed_list_key] = num_elements[list_key]

# do the same thing with values now
all_value_keys = sorted(values.keys(), key=lambda index: len(index))
fixed_values = defaultdict(lambda: set())
for value_key in all_value_keys:
    fixed_value_key = value_key
    for real_list_key in real_list_keys:
        if fixed_value_key.startswith(real_list_key):
            fixed_value_key = re.sub(rf'({real_list_key}),\d+(,?.*)', r'\1,i\2', fixed_value_key)
    if fixed_value_key in fixed_values:
        fixed_values[fixed_value_key] |= values[value_key]
    else:
        fixed_values[fixed_value_key] = values[value_key]

# for value_key in sorted(fixed_values.keys(), key=lambda index: len(index)):
for value_key in sorted(fixed_values.keys()):
    values = fixed_values[value_key]
    # if all([len(value) > 100 if isinstance(value, str) else True for value in values]):
        # print(f'{value_key} looks like jumbled data')
        # continue
    
    print(f'{value_key}: {", ".join(map(str, fixed_values[value_key]))}')

found real list 0,i,0,1
found real list 0,i,0,2
found real list 0,i,0,5
found real list 0,i,0,8
found real list 0,i,0,13
found real list 0,i,0,24
found real list 0,i,0,2,i,8
found real list 0,i,0,22,9
found real list 0,i,0,2,i,12
found real list 0,i,0,2,i,10
found real list 0,i,0,2,i,18
0,i,0,0: UA, multi, AA, DL
0,i,0,1,i: Avianca, Gol, United, American, LATAM, COPA, Air Canada, Alaska, Aeromexico, Delta
0,i,0,10: 1, 2
0,i,0,11: None
0,i,0,12: False
0,i,0,13,i,0: 515, 520, 523, 524, 25, 540, 29, 1053, 31, 550, 40, 42, 43, 556, 44, 47, 559, 50, 563, 52, 54, 567, 55, 57, 1079, 60, 61, 70, 74, 75, 78, 79, 80, 81, 82, 595, 86, 600, 90, 91, 605, 94, 95, 98, 102, 103, 105, 621, 110, 111, 624, 109, 623, 115, 119, 120, 634, 123, 126, 127, 640, 130, 132, 134, 135, 648, 137, 143, 146, 149, 150, 156, 157, 160, 164, 165, 170, 683, 174, 176, 177, 690, 180, 181, 185, 186, 190, 194, 197, 714, 205, 209, 210, 211, 215, 217, 224, 225, 230, 234, 237, 762, 254, 258, 260, 263, 268, 269, 271, 1301, 281, 28

Now, reverse engineering the indices should be much easier using `fixed_values`. Define more search pairs to get a wider set of data

In [109]:
root_keys = {
    0: {},
    1: {
        'name': 'to_from_details',
        0: {
            0: {
                'name': 'departure_airport_details',
            },
            1: {
                'name': 'arrival_airport_details',
            },
        },
    },
    2: {
        0: {
            'name': 'best_flights',
            'list': True,
            'type': 'trip',
        },
    },
    3: {
        0: {
            'name': 'other_flights',
            'list': True,
            'type': 'trip',
        },
    }
}
keys = {
    0: {
        'name': 'trip',
        0: 'airline',
        1: {
            0: 'airline_name',
        },
        2: {
            'name': 'flight_leg',
            'i': {
                2: 'operator',
                3: 'departure_airport',
                4: 'departure_airport_name',
                5: 'arrival_airport_name',
                6: 'arrival_airport',
                8: {
                    0: 'DEPART_HOUR',
                    1: 'DEPART_MIN',
                },
                10: {
                    0: 'ARRIVE_HOUR',
                    1: 'ARRIVE_MIN',
                },
                14: 'seat_pitch',
                15: {
                    'name': 'codeshares',
                    0: {
                        0: 'airline',
                        1: 'flight_number',
                        3: 'airline_name',
                    },
                },
                17: 'aircraft',
                20: {
                    0: 'DEPART_YEAR',
                    1: 'DEPART_MONTH',
                    2: 'DEPART_DAY',
                },
                21: {
                    0: 'ARRIVE_YEAR',
                    1: 'ARRIVE_MONTH',
                    2: 'ARRIVE_DAY',
                },
                22: {
                    0: 'airline',
                    1: 'flight_number',
                    3: 'airline_name',
                },
            },
        },
        3: 'departure_airport',
        4: {
            0: 'DEPART_YEAR',
            1: 'DEPART_MONTH',
            2: 'DEPART_DAY',
        },
        5: {
            0: 'DEPART_HOUR',
            1: 'DEPART_MIN',
        },
        6: 'arrival_airport',
        7: {
            0: 'ARRIVE_YEAR',
            1: 'ARRIVE_MONTH',
            2: 'ARRIVE_DAY',
        },
        8: {
            0: 'ARRIVE_HOUR',
            1: 'ARRIVE_MIN',
        },
        9: 'price',
        13: {
            'name': 'LAYOVER',
            'i': {
                0: 'MINUTES',
                1: 'DEPART_AIRPORT_ICAO',
                2: 'ARRIVE_AIRPORT_ICAO',
                4: 'DEPART_AIRPORT_NAME',
                5: 'DEPART_AIRPORT_CITY',
                6: 'ARRIVE_AIRPORT_NAME',
                7: 'ARRIVE_AIRPORT_CITY',
            },
        },
        22: ['EMISSIONS_DATA'],
    },
}

In [6]:
for result in results:
    print(result)

Other Flights:
Delta flight ($77) with 0 stops from LAS at 2024-12-31 06:00:00 to LAX at 2024-12-31 07:17:00
United flight ($78) with 0 stops from LAS at 2024-12-31 06:02:00 to LAX at 2024-12-31 07:20:00
United flight ($78) with 0 stops from LAS at 2024-12-31 20:30:00 to LAX at 2024-12-31 21:48:00
Alaska flight ($81) with 0 stops from LAS at 2024-12-31 17:54:00 to LAX at 2024-12-31 19:15:00
United flight ($78) with 0 stops from LAS at 2024-12-31 15:03:00 to LAX at 2024-12-31 16:21:00
United flight ($78) with 0 stops from LAS at 2024-12-31 10:37:00 to LAX at 2024-12-31 11:55:00
United flight ($396) with 1 stops from LAS at 2024-12-31 17:30:00 to LAX at 2025-01-01 00:06:00
Southwest flight ($485) with 2 stops from LAS at 2024-12-31 08:10:00 to LAX at 2024-12-31 16:15:00
Alaska flight ($99) with 0 stops from LAS at 2024-12-31 06:00:00 to LAX at 2024-12-31 07:39:00
American flight ($77) with 0 stops from LAS at 2024-12-31 07:00:00 to LAX at 2024-12-31 08:17:00
American flight ($79) with 0 